In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,492.20,5.1990,3414.093682,41.97,NaN,NaN
ENDOFVD,210.00,0.5490,2874.837567,41.97,NaN,NaN
DROSND-TURBND,191.90,0.4000,2843.887142,12.88,NaN,NaN
SMESHEND,78.16,0.4000,327.573430,54.85,NaN,NaN
DOOTB2,131.40,0.2542,2724.861533,54.85,NaN,NaN
DOOTB1,NaN,0.1525,2686.016192,54.85,NaN,NaN
INCND,NaN,0.1525,2686.016192,54.85,NaN,NaN
INKOND,NaN,0.0048,2341.001493,54.85,NaN,NaN


In [2]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)

Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0, maxiterations=50, calctolerance=10**-3)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

Максимальная погрешность определения давления в отборах 0.0005801064546580738
fin:--- 0.1 сек. ---


{'Max_error': 0.0005801064546580738,
 'P_out': [5.189007902472844,
  0.41470762366614,
  0.47670762366613995,
  0.41470762366614,
  0.2334491870372846,
  0.1060254894205336,
  0.1060254894205336,
  0.0048],
 'Eff_out': [0.8493807611371985,
  0.8372415159994411,
  0.8149716606831633,
  0.7339885635728762]}

In [3]:
# надо посчитать потерю давления НД (решаю)
# отборы неправильно
water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,492.149515,5.189008,3414.093682,41.97,6.934924,100.00000
ENDOFVD,199.803382,0.414708,2859.773322,41.97,7.153557,100.00000
DROSND-TURBND,193.852969,0.476708,2843.887142,12.88,7.057021,100.00000
SMESHEND,198.031482,0.414708,2856.042893,54.85,7.145654,100.00000
DOOTB2,147.098533,0.233449,2760.409491,54.85,7.190363,100.00000
DOOTB1,101.249727,0.106025,2647.456554,54.85,7.258856,98.66491
INCND,101.249727,0.106025,2647.456554,54.85,7.258856,98.66491
INKOND,32.149642,0.004800,2325.258354,54.85,7.641335,90.34263


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000
INCND,NaN,0.1525,2652.000000,78.16,NaN
INKOND,NaN,0.0080,2338.024560,78.16,NaN
